In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import openai
import os
import json
from tqdm import tqdm
from tabqa.GptPrompter import *
from tabqa.GptCOTPrompter import *
from tabqa.GptCOTPrompter_BeamSeach import *
from tabqa.AutoReasoner import *
import dotenv

config = dotenv.dotenv_values("../.env")
openai.api_key = config['OPENAI_API_KEY']
openai.base_url = config['GPT_BASE_URL']

dataset = pd.read_csv('../dataset/WikiTableQuestions/data/pristine-unseen-tables.tsv', sep='\t')

TypeError: __init__() got an unexpected keyword argument 'proxies'

In [3]:
NNDemo = False
max_demo = 5
gpt_model = 'gpt-4'

program = 'sql-py'
template = 'original-sql-py-no-intermediate'

def parallel_codex_func_formatv1(i):
    max_retry = 3
    while max_retry>0:
        try:
            codex_prompter = CodexAnswerCOTExecutor_HighTemperaturMajorityVote(
                                                f'prompt_template/{template}.json',
                                                dataset.iloc[i]['id'], 
                                                dataset.iloc[i]['utterance'], 
                                                dataset.iloc[i]['context'], 
                                                dataset.iloc[i]['targetValue'],  
                                                base_path='../dataset/WikiTableQuestions/',
                                                demo_file=f'few-shot-demo/WikiTQ-{program}.json',
                                                )
            codex_prompter.max_demo = max_demo
            codex_prompter.model = gpt_model
            codex_prompter._gen_gpt_prompt(NNDemo)
            codex_prompter._get_gpt_prediction_majority_vote(repeat_times=5)
            log = codex_prompter._log_dict()
            break
        except Exception as e:
            log = {
                'id': dataset.iloc[i]['id'],
                'uncaught_err': str(e)
            }
            if "model's maximum context length" in str(e):
                return log
            max_retry -= 1
    return log
    
n_threads = 3
maxLimit = 5

from joblib import Parallel, delayed
output_result_file = f'../dataset/WikiTableQuestions/results/CodexAnswerCOTExecutor_HighTemperaturMajorityVote_{template}_{program}_NNDemo={NNDemo}_results_pristine-unseen-tables_limit{maxLimit}_model{gpt_model}.json'
logs = Parallel(n_jobs=n_threads, require='sharedmem')(delayed(parallel_codex_func_formatv1)(i) for i in tqdm(range(min(maxLimit, dataset.shape[0]))))    
json.dump(logs, open(output_result_file, 'w'), indent=4)

100%|██████████| 5/5 [00:00<00:00, 324.02it/s]


In [4]:
# evaluate: 
os.system(f'cd ../dataset/WikiTableQuestions/ && python2 evaluator.py ./results/{output_result_file.split("/")[-1]} ')

Reading dataset from ./tagged/data/training.tagged
Reading dataset from ./tagged/data/pristine-seen-tables.tagged
Reading dataset from ./tagged/data/pristine-unseen-tables.tagged
Read 22033 examples
Reading predictions from ./results/CodexAnswerCOTExecutor_HighTemperaturMajorityVote_original-sql-py-no-intermediate_sql-py_NNDemo=False_results_pristine-unseen-tables_limit5_modelgpt-4.json
Examples: 5
Correct: 4
Accuracy: 0.8
GPT error: 0.0
Uncaught error: 0.0
{}


0